In [ ]:
from mysql_conn import mysql_connection
import nltk
import re
nltk.download('punkt')

In [ ]:
db = mysql_connection()
cursor = db.cursor

# Create table
sql = """
CREATE TABLE ilan_degisen
(
    ilan_no INT,
    degisen INT
)
"""

try:
    cursor.execute(sql)
except:
    print("Table already exists!")
    
    
# Create table
sql = """
CREATE TABLE ilan_boya
(
    ilan_no INT,
    boya INT
)
"""

try:
    cursor.execute(sql)
except:
    print("Table already exists!")
    
    
# Create table
sql = """
CREATE TABLE ilan_tramer
(
    ilan_no INT,
    tramer INT
)
"""

try:
    cursor.execute(sql)
except:
    print("Table already exists!")
    
# Create table
sql = """
CREATE TABLE ilan_hts
(
    ilan_no INT,
    degisen INT,
    boya INT,
    tramer INT
)
"""

try:
    cursor.execute(sql)
except:
    print("Table already exists!")

cursor.close()

In [ ]:
def insert_db(table_name, column_name, db, ilan_no, cursor, num):

    sql = "INSERT INTO " + table_name + " (ilan_no, " +  column_name + ") VALUES (%s, %s)"

    val = (ilan_no, num)
    cursor.execute(sql, val)

    db.mydb.commit()

In [ ]:
def insert_query(key, key_arr, column_name, table_name, debug_flag = False):
    
    # Connect to database
    db = mysql_connection()
    cursor = db.cursor
    
    # Select Query
    
    if debug_flag:
        cursor.execute("SELECT ilan_no, aciklama, diger FROM ilan_detay")
    else:
        cursor.execute("SELECT ilan_no, aciklama, diger FROM ilan_detay WHERE ilan_no NOT IN (SELECT ilan_no FROM " + table_name + ")")
    
    result = cursor.fetchall()
    
    # For debug
    all_ext_words = []
    all_sentences = []
    
    for row in result:

        ilan_no = row[0]
        aciklama = row[1]
        diger = row[2]
        
        
        if not debug_flag:
        
            # Filter directly 

            # All are original (not trusted)
            if 'Aracın tüm parçaları orijinaldır' in diger:
                pass

            # No information
            elif not diger:
                pass

            # Only vehicle parts are original (not trusted) so only insert 1 to boya
            elif ('Aracın parçaları orijinaldir' in diger) and (key == "boya"):
                insert_db(table_name, column_name, db, ilan_no, cursor, 1)       
                continue

            # Only vehicle paint is original (not trusted) so only insert 1 to degis
            elif ('Aracın boyası orijinaldir' in diger) and (key == "değiş"): 
                insert_db(table_name, column_name, db, ilan_no, cursor, 1)
                continue

            # Both are not original
            elif key == "boya" or key == "değiş":
                # insert 1 to column in ilan_hts
                insert_db(table_name, column_name, db, ilan_no, cursor, 1)
                continue
            
        # Split into sentences
        sentences = nltk.sent_tokenize(aciklama)
        
        ext_words = []
        end_indexes = []
        inc_sentences = []
        
        for sentence in sentences:
            sentence = sentence.lower()

            # Correct encoding
            enc = sentence.encode('iso-8859-9',errors='replace')
            dec = enc.decode('iso-8859-9')
            sentence = dec.replace("?", "")
            sentence = sentence.replace("\xa0", " ") 
            
            # Search the key in sentences
            if key in sentence:
                inc_sentences.append(sentence)
                # Extract the word
                for m in re.finditer(key, sentence):  
                    start_ind = m.start()
                    word_len = sentence[start_ind:].find(" ")
                    end_index = start_ind + word_len
                    ext_word = sentence[start_ind:end_index]
                    
                    ext_words.append(ext_word)
                    end_indexes.append(end_index)
                        
                    # Debug
                    if ext_word not in all_ext_words:
                        all_ext_words.append(ext_word)
                
                all_sentences.append(sentence)
                        
        if not debug_flag:
            # If there is extracted words
            if ext_words:

                skip_flag = False
                for word in key_arr['bad']:
                    for ext_word in ext_words:
                        if word in ext_word:

                            # insert 1 to column in ilan_hts
                            insert_db(table_name, column_name, db, ilan_no, cursor, 1)

                            skip_flag = True
                            break

                    if skip_flag == True:
                        break

                # We already inserted a value so skip remaining checks for the same ilan_no
                if skip_flag == True:
                    continue            

                for word in key_arr['good']:
                    for ext_word in ext_words:
                        if word in ext_word: 
                            # insert 0 to column in ilan_hts
                            insert_db(table_name, column_name, db, ilan_no, cursor, 0)

                            skip_flag = True
                            break

                    if skip_flag == True:
                        break

                # We already inserted a value so skip remaining checks for the same ilan_no
                if skip_flag == True:
                    continue            


                skip_flag3 = False

                for word in key_arr['unsure']:
                    for ext_word, end_index in zip(ext_words, end_indexes):
                        if word in ext_word:       
                            
                            skip_flag4 = False
                            for sentence in inc_sentences:
                        
                                skip_flag2 = False

                                new_sentence,_ = find_spaces_after(sentence, end_index, 10)

                                for word in key_arr['unsure-bad']:
                                    if word in new_sentence:
                                        # insert 1 to column in ilan_hts
                                        insert_db(table_name, column_name, db, ilan_no, cursor, 1)

                                        skip_flag2 = True
                                        break

                                if skip_flag2 == True:
                                    skip_flag4 = True
                                    break

                                for word in key_arr['unsure-good']:
                                    if word in new_sentence:
                                        # insert 0 to column in ilan_hts
                                        insert_db(table_name, column_name, db, ilan_no, cursor, 0)

                                        skip_flag2 = True
                                        break

                                if skip_flag2 == True:
                                    skip_flag4 = True
                                    break
                            
                            if skip_flag4 == True:
                                skip_flag3 = True
                                break

                    if skip_flag3 == True:
                        break

                # Nothing left to filter probably it is there

                # insert 1 to column in ilan_hts
                if skip_flag3 == False:
                    insert_db(table_name, column_name, db, ilan_no, cursor, 1)

            # There is not enough information in sentence
            else:
                # insert 0 to degisen in ilan_hts 
                insert_db(table_name, column_name, db, ilan_no, cursor, 0)
    
    # Close db
    cursor.close()
    
    # For debugging
    return all_ext_words,all_sentences    

In [ ]:
def find_spaces_before(sent, key, filter_len):
    
    key_index = sent.find(key) + len(key)
    
    space_indexes = []
    
    for m in re.finditer(" ", sent[:key_index]):
        space_indexes.append(m.start())                                

    len_space = len(space_indexes)

    ind = 0
    if filter_len < len_space:
        ind = len_space-filter_len

    if space_indexes:
        sent_start_index = space_indexes[ind]
    else:
        sent_start_index = 0  

    return sent[sent_start_index:key_index], key_index 

In [ ]:
def find_spaces_after(sentence, end_index, filter_len):
    sentence = sentence + " "
    indexes = []
    
    for m in re.finditer(" ", sentence[end_index+1:]):
        indexes.append(m.start()) 

    len_index = len(indexes)

    if filter_len > len_index:
        filter_len = len_index

    sent_last_index = end_index + 1 + indexes[filter_len-1]

    return sentence[end_index+1: sent_last_index], sent_last_index

In [ ]:
def find_prices(sent):
    
    reg = re.findall('\d+(?:\.\d{3})', sent)

    if reg:
        for i,r in enumerate(reg):
            reg[i] = int(r.replace(".",""))

        return max(reg)

    else:
        reg = re.findall('\d+', sent)
        if reg:
            for i,r in enumerate(reg):
                reg[i] = int(r)
                
            price = max(reg)
            if price < 1000:
                price= price*1000
            return price

In [ ]:
def insert_number_query(key, key_arr, column_name, table_name):
    
    # Connect to database
    db = mysql_connection()
    cursor = db.cursor
    
    # Select Query
    cursor.execute("SELECT ilan_no, aciklama FROM ilan_detay WHERE ilan_no NOT IN (SELECT ilan_no FROM " + table_name + ")")
    result = cursor.fetchall()
    
    for row in result:

        ilan_no = row[0]
        aciklama = row[1]
      
        # Split into sentences
        sentences = nltk.sent_tokenize(aciklama)
        
        ext_words = []
        end_indexes = []
        inc_sentences = []
        for sentence in sentences:
            sentence = sentence.lower()

            # Correct encoding
            enc = sentence.encode('iso-8859-9',errors='replace')
            dec = enc.decode('iso-8859-9')
            sentence = dec.replace("?", "")
            sentence = sentence.replace("\xa0", " ") 

            # Search the key in sentences
            if key in sentence:
                inc_sentences.append(sentence)
                # Extract the word
                for m in re.finditer(key, sentence):  
                    start_ind = m.start()
                    word_len = sentence[start_ind:].find(" ")
                    end_index = start_ind + word_len
                    ext_word = sentence[start_ind:end_index]

                    
                    ext_words.append(ext_word)
                    end_indexes.append(end_index)
                        
        # If there is extracted words
        if ext_words:
            skip_flag = False   

            for word in key_arr['good']:
                for ext_word in ext_words:
                    if word in ext_word: 
                        # insert 0 to column in ilan_hts
                        
                        insert_db(table_name, column_name, db, ilan_no, cursor, 0)
                        skip_flag = True
                        break

                if skip_flag == True:
                    break

            # We already inserted a value so skip remaining checks for the same ilan_no
            if skip_flag == True:
                continue            
                        
                    
            skip_flag3 = False

            for word in key_arr['unsure']:
                for ext_word, end_index in zip(ext_words, end_indexes):
                    if word in ext_word:       

                        skip_flag4 = False
                        for sentence in inc_sentences:

                            skip_flag2 = False                            
                                           
                            new_sentence, sent_last_index = find_spaces_after(sentence, end_index, 10)

                            for word in key_arr['unsure-good']:
                                if word in new_sentence:
                                    # insert 0 to column in ilan_hts
                                    
                                    insert_db(table_name, column_name, db, ilan_no, cursor, 0)
                                    skip_flag2 = True
                                    break

                            if skip_flag2 == True:
                                skip_flag4 = True
                                break
                            
                            space_indexes = []
                            for m in re.finditer(" ", sentence[:end_index]):
                                space_indexes.append(m.start())
                            
                            filter_len = 10
                            len_space = len(space_indexes)
                            
                            ind = 0
                            if filter_len < len_space:
                                ind = len_space-filter_len
                            
                            if space_indexes:
                                sent_start_index = space_indexes[ind]
                            else:
                                sent_start_index = 0
                            
                            
                            new_sent = sentence[sent_start_index:sent_last_index]
                            
                            # Check prices
                            price  = 0
                            price1 = 0
                            prices = []
                            
                            if " tl" in new_sent and " bin " in new_sent and new_sent.count(" tl") == 1:
                                
                                tl_new_sent,_ = find_spaces_before(new_sent, "tl", 5)
                                                          
                                price = int(max(re.findall('\d+', tl_new_sent)))
                                if price < 1000:
                                    price = price*1000
                                    
                                #insert price
                                insert_db(table_name, column_name, db, ilan_no, cursor, price)
                                skip_flag4 = True
                                break
                                

                            elif " tl" in new_sent or "0tl" in new_sent:
                                                    
                                loop_sent = new_sent
                                
                                while loop_sent.find("tl") != -1:
                                
                                    tl_new_sent,tl_index = find_spaces_before(loop_sent, "tl", 5)
                                    
                                    price1 = find_prices(tl_new_sent)
                                    
                                    if type(price1) == int :
                                        prices.append(price1)
                                
                                    loop_sent = loop_sent[tl_index:]
                                
                                price = max(prices)
                                  
                                #insert price
                                insert_db(table_name, column_name, db, ilan_no, cursor, price)
                                skip_flag4 = True
                                break             
                                
                            elif " tl" not in new_sent and "km" in new_sent:
                                
                                #insert 0
                                insert_db(table_name, column_name, db, ilan_no, cursor, 0)
                                skip_flag4 = True
                                break                                
                                
                            # If there is digit
                            elif True in [c.isdigit() for c in new_sent]:
                                
                            
                                
                                tramer_new_sent_bf,key_index = find_spaces_before(new_sent, "tramer", 5)
                                tramer_new_sent_af,_ = find_spaces_after(new_sent, key_index,5)
                                
                                
                                price_bf = find_prices(tramer_new_sent_bf)
                                price_af = find_prices(tramer_new_sent_af)
                                
                                                             
                                if price_bf and price_af:
                                    
                                
                                
                                    if price_bf > price_af:
                                        price1 = price_bf
                                    else:
                                        price1 = price_af    
                                elif price_bf:                                 
                                    price1 = price_bf
                                elif price_af:
                                    price1 = price_af                                
                                
                                
                                if type(price1) == int:
                                    prices.append(price1)
                                    
                                price = max(prices)
                                
                                #insert price
                                insert_db(table_name, column_name, db, ilan_no, cursor, price)
                                skip_flag4 = True
                                break     
                                
                        if skip_flag4 == True:
                            skip_flag3 = True
                            break

                if skip_flag3 == True:
                    break

            # Nothing left to filter probably it is not there

            # insert 0 to column in ilan_hts
            if skip_flag3 == False:     
                insert_db(table_name, column_name, db, ilan_no, cursor, 0)
        # There is not enough information in sentence
        else:
            # insert 0 to degisen in ilan_hts 
            insert_db(table_name, column_name, db, ilan_no, cursor, 0)
    
    # Close db
    cursor.close()

In [ ]:
degis = {
            "bad": ["değişmiş", "değişti", "değişik", "değişerek", "değişenli", "değişendir"],
            "good": ["değişensiz"],
            "unsure": ["değişen", "değişme", "değişim"],
            "unsure-bad": ["var", "bulunmak", "mevcut", "kaput", "kapi", "kapı", "çamur", "camur", "tampon"],
            "unsure-good" : ["yok", "olma", "bulunmama", "bulunmay"]

        }

boya = {
            "bad": ["boyal", "boyanm", "boyand"],
            "good": ["boyas"],
            "unsure": ["boya"],
            "unsure-bad": ["var", "bulunmak", "mevcut"],
            "unsure-good" : ["yok", "olma", "bulunmama", "bulunmay"]

        }
tramer = {
            "good": ["tramers"],
            "unsure": ["tramer"],
            "unsure-good" : ["yok", "olma", "bulunmama", "bulunmay"]

        }

In [ ]:
# Run continuously in server
while True:
    
    key = "değiş"
    column_name = "degisen"
    table_name = "ilan_degisen"
    
    insert_query(key, degis, column_name, table_name)
    
    key = "boya"
    column_name = "boya"
    table_name = "ilan_boya"
    
    insert_query(key, boya, column_name, table_name)
    
    key = "tramer"
    column_name = "tramer"
    table_name = "ilan_tramer"
    
    insert_number_query(key, tramer, column_name, table_name)
    

    # join all 3 table in ilan_hts table

    db = mysql_connection()
    cursor = db.cursor

    cursor.execute("CREATE TABLE temp1 AS (SELECT t1.ilan_no, degisen, boya FROM ilan_degisen t1 INNER JOIN ilan_boya t2 ON t1.ilan_no = t2.ilan_no)")
    db.mydb.commit()

    cursor.execute("CREATE TABLE temp2 AS (SELECT t1.ilan_no, degisen, boya, tramer FROM temp1 t1 INNER JOIN ilan_tramer t2 ON t1.ilan_no = t2.ilan_no)")
    db.mydb.commit()

    sql = """
    INSERT INTO ilan_hts(ilan_no, degisen, boya, tramer)

    SELECT temp2.ilan_no,temp2.degisen,temp2.boya,temp2.tramer

    FROM temp2

    WHERE NOT EXISTS (
        SELECT *
        FROM ilan_hts 
        WHERE ilan_hts.ilan_no=temp2.ilan_no
    )

    """

    cursor.execute(sql)

    cursor.execute("DROP TABLE temp1")
    cursor.execute("DROP TABLE temp2")
    db.mydb.commit()